In [143]:
import json
# data = json.load(open('output_dialog_missing_1.json', 'r'))
data = json.load(open('output_dialog_tgredial.json', 'r'))

In [44]:
test_data = json.load(open('tgredial_bert/test_data.json', 'r'))

In [47]:
test_data[0]['messages'][-1]['final']

['反馈，结束', [None]]

In [101]:
# Get the mentioned movies
# with multiple movies
# list(data[0].keys())[0].split(' ')
# with individual movies
list(data[2].keys())[0]

'我知道（1956年中国动电影）'

In [144]:
entity2id = json.load(open('tgredial_bert/entity2id.json', 'r'))

In [148]:
entity2id[list(data[0].keys())[0]]

1

In [139]:
import re
# Get the utterances
# list(data[0].values())[0]['choices'][0]['message']['content'].split('\n')
text = list(data[6].values())[0]
text = text.replace("\n", "")
print(text)
# (r'\s(?=\[用户\]:|\[系统\]:)
# result = re.split(r'(\[用户\]:|\[系统\]:)', text)

# result = re.findall(r'\[(用户|系统)\]:\s(.*?)((?=\[(用户|系统)\]:)|$)', text)

# # Combining the tag and message into a single string
# result = ['[' + tag + ']: ' + message for tag, message, _, _ in result]

# Split the text into strings starting with [用户] or [系统]
strings = re.split(r'\s*(?=\[用户\]|\[系统\])', text)
# Remove empty strings from the list
strings = [s.strip() for s in strings if s.strip()]

[用户]: 嗨，我正在寻找一部好电影看。你有什么推荐吗？ [系统]: 当然！你有任何特定类型或题材的电影偏好吗？ [用户]: 我想看一部关于中国历史的战争电影。 [系统]: 好的，如果你对抗日战争感兴趣，我可以向你推荐一部叫做《大决战之平津战役》的电影。这是一部讲述中国人民解放军在抗日战争中抵御日军入侵的电影。 [用户]: 听起来不错！请告诉我更多关于《大决战之平津战役》的信息。 [系统]: 《大决战之平津战役》是由中国著名导演冯小刚执导的一部战争电影。它以中国人民解放军在抗日战争中抵御日军入侵的历史事件为背景，讲述了中国军队在平津战役中对抗日军所展现出的英勇和坚韧。影片中揭示了中国人民在艰苦环境下崭露头角的精神面貌，以及中国军队在历史上的无畏精神。 [用户]: 这部电影听起来很激动人心！我一定会去看看。 [系统]: 是的，这部电影确实很值得一看。希望你能够喜欢这个推荐。如果你还有其他问题，请随时告诉我！


In [140]:
strings

['[用户]: 嗨，我正在寻找一部好电影看。你有什么推荐吗？',
 '[系统]: 当然！你有任何特定类型或题材的电影偏好吗？',
 '[用户]: 我想看一部关于中国历史的战争电影。',
 '[系统]: 好的，如果你对抗日战争感兴趣，我可以向你推荐一部叫做《大决战之平津战役》的电影。这是一部讲述中国人民解放军在抗日战争中抵御日军入侵的电影。',
 '[用户]: 听起来不错！请告诉我更多关于《大决战之平津战役》的信息。',
 '[系统]: 《大决战之平津战役》是由中国著名导演冯小刚执导的一部战争电影。它以中国人民解放军在抗日战争中抵御日军入侵的历史事件为背景，讲述了中国军队在平津战役中对抗日军所展现出的英勇和坚韧。影片中揭示了中国人民在艰苦环境下崭露头角的精神面貌，以及中国军队在历史上的无畏精神。',
 '[用户]: 这部电影听起来很激动人心！我一定会去看看。',
 '[系统]: 是的，这部电影确实很值得一看。希望你能够喜欢这个推荐。如果你还有其他问题，请随时告诉我！']

In [21]:
from fuzzywuzzy import fuzz
def get_max_sim_score(uttrs, movie_name):
    max_score = 0 
    for uttr in uttrs:
        score = fuzz.partial_ratio(uttr, movie_name)
        if score > max_score:
            max_score = score
    return max_score
    
def fuzzy_match(uttr, movie_name, max_score):
    # print(fuzz.partial_ratio(uttr, movie_name))
    if fuzz.partial_ratio(uttr, movie_name) == max_score:
        return True
    else:
        return False

movie_name = list(data[1].keys())[0]

max_score = get_max_sim_score(example, movie_name)
print(max_score)
for uttr in example:
    print(fuzzy_match(uttr, movie_name, max_score))

100
False
False
False
True
True
True
False
False


In [43]:
from LAC import LAC
lac = LAC(mode='rank')
for uttr in example:
    result = lac.run(uttr[6:].strip())
    words = result[0]
    scores = result[2]
    for idx in range(len(scores)):
        if scores[idx] >1:
            print(words[idx])
            
    print("---------------------------")

嗨
寻找
一部
好
电影
推荐
---------------------------
特定
类型
题材
电影偏好
---------------------------
一部
全家
动画片
推荐
---------------------------
一部
全家
动画片
我可以
一部
叫做
精灵鼠小弟
电影
电影
E·B·怀特
童话
改编
一个
小老鼠
朋友们
人类
家庭
战胜
恶猫
故事
电影
非常
全年龄段
---------------------------
起来
不错
精灵鼠小弟
---------------------------
精灵鼠小弟
一部
1999年
好莱坞
动画
梦工厂
罗伯特·泽米吉斯
执导
电影
好评
赢得
奖项
奥斯卡
最佳
动画
长片
提名
声音
阵容
大卫·斯佩德
霍兰德·泰勒
茱莉亚·路易斯-德瑞弗斯和约拿森·利普尼基
一部
非常
动画片
全家
一起
欣赏
---------------------------
好
谢谢
推荐
现在
去
---------------------------
不用
客气
精灵鼠小弟
随时
告诉我
---------------------------


### Utterance Process

In [151]:
from fuzzywuzzy import fuzz
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from LAC import LAC
lac = LAC(mode='rank')

def get_max_sim_score(uttrs, movie_name):
    max_score = 0 
    for uttr in uttrs:
        score = fuzz.partial_ratio(uttr, movie_name)
        if score > max_score:
            max_score = score
    return max_score

def fuzzy_match(uttr, movie_name, max_score):
    # print(fuzz.partial_ratio(uttr, movie_name))
    if fuzz.partial_ratio(uttr, movie_name) > 60:
        return True
    else:
        return False

# def fuzzy_replace(str_a, str_b, orig_str):
#     l = len(str_a.split()) # Length to read orig_str chunk by chunk
#     splitted = orig_str.split()
#     for i in range(len(splitted)-l+1):
#         test = " ".join(splitted[i:i+l])
#         if fuzz.partial_ratio(str_a, test) > 60: #Using fuzzwuzzy library to test ratio
#             before = " ".join(splitted[:i])
#             after = " ".join(splitted[min(len(splitted), i+l):])
#             return before+" "+str_b+" "+after #Output will be sandwich of these three strings


def utterance_convert(uttr, movie, max_score):
    if uttr[:4] == '[用户]':
        role = 'Seeker'
    elif uttr[:4] == '[系统]':
        role = 'Recommender'
    else:
        role = None
    text = list(uttr[6:].strip())
    
    movie_mentioned = []
    
    if fuzzy_match(uttr, movie, max_score):
        movie_mentioned.append(movie)
    result = lac.run(uttr[6:].strip())
    if len(result) < 3:
        print("uttr: ", uttr)
        print(result)
    split_word = result[0]
    scores = result[2]
    entity = []
    word = []
    target = []
    for idx in range(len(scores)):
        if scores[idx] > 2:
            target.append(split_word[idx])
        if scores[idx] > 1:
            entity.append(split_word[idx])
        if scores[idx] > 0:
            word.append(split_word[idx])
    return role, text, movie_mentioned, entity, word, target

# # Test instance
# movies = ['<http://dbpedia.org/resource/Her_Battle_for_Existence>', '<http://dbpedia.org/resource/12_Rounds_(film)>','<http://dbpedia.org/resource/The_Three_Musketeers_(1986_film)>']
# movie_ids = {'<http://dbpedia.org/resource/Her_Battle_for_Existence>':101245, '<http://dbpedia.org/resource/12_Rounds_(film)>':123, '<http://dbpedia.org/resource/The_Three_Musketeers_(1986_film)>':456}
# uttr1 = "What kind of movies are you in the mood for?"
# uttr2 = "Sure, how about 'Her Battle for Existence?' It\'s a documentary about a woman\'s journey to overcome a rare and life-threatening disease.'"
# uttr3 = "You might enjoy '12 Rounds,' which is an action-packed thriller about a cop trying to save his kidnapped girlfriend."
# uttr4 = "Have you seen 'The Three Musketeers' from 1986? It\'s an adventure movie set in 17th century France about three musketeers who protect the king."
# utterance_convert(uttr4, movies, movie_ids)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [152]:
import pandas as pd
from fuzzywuzzy import fuzz
import tqdm
from datetime import datetime
from collections import defaultdict

def cal_sim(name):
    return fuzz.ratio(movie, name)

def name_to_link(name, movie_ids):
    movie_ids['sim'] = movie_ids['movieName'].apply(lambda x: fuzz.ratio(name, x))
    return movie_ids[movie_ids['sim'] == movie_ids['sim'].max()]

def check_empty(uttr):
    if len(uttr.strip()) == 0:
        return False
    elif len(uttr[6:].strip()) == 0:
        return False
    return True

# syn_data = []
syn_data = defaultdict(dict)

for conv_idx, instance in enumerate(data):
    if conv_idx % 1000 == 0:
        print("progress: ", conv_idx / len(data) * 100, "%", datetime.now())
    syn_data_inst = {}
    
    # assign conversation id     
    conv_id = 'syn_' + str(conv_idx)
    syn_data_inst['conv_id'] = conv_id
    syn_data_inst['user_id'] = 'syn'
    syn_data_inst['messages'] = []
    dialog_text = list(instance.values())[0]
    dialog_text = dialog_text.replace("\n", "")
    strings = re.split(r'\s*(?=\[用户\]|\[系统\])', dialog_text)
    # Remove empty strings from the list
    utterances = [s.strip() for s in strings if s.strip()]
    
    # utterances = re.split(r'\n\n|\n', list(instance.values())[0])
    utterances = list(filter(check_empty, utterances))
    utterances = list(filter(None, utterances))
    
    movie_name = list(instance.keys())[0]
    max_score = get_max_sim_score(utterances, movie_name)
    
    broken_dialog = False
    for idx, utterance in enumerate(utterances):
        uttr_info = {}
        utt_id = idx
        role, text, movie_mentioned, entity, word, target = utterance_convert(utterance, movie_name, max_score)
        if role is None: 
            broken_dialog = True
        if broken_dialog:
            break
        else:
            uttr_info['local_id'] = idx + 1 
            uttr_info['role'] = role
            uttr_info['text'] = text
            uttr_info['target'] = target
            uttr_info['word'] = word
            uttr_info['entity'] = entity
            uttr_info['movie'] = movie_mentioned
            if idx == len(utterances) - 1:
                uttr_info['final'] = ['反馈，结束', [None]]
            else:
                uttr_info['final'] = ['请求推荐', target]
                
            syn_data_inst['messages'].append(uttr_info)
    if broken_dialog:
        continue
    syn_data[str(entity2id[list(instance.keys())[0]])] = syn_data_inst

progress:  0.0 % 2023-06-14 09:34:04.041388
progress:  3.1264655307175238 % 2023-06-14 09:34:32.355089
progress:  6.2529310614350475 % 2023-06-14 09:35:00.626752
progress:  9.37939659215257 % 2023-06-14 09:35:29.026509
progress:  12.505862122870095 % 2023-06-14 09:35:57.197923
progress:  15.63232765358762 % 2023-06-14 09:36:25.879486
progress:  18.75879318430514 % 2023-06-14 09:36:54.110523
progress:  21.88525871502267 % 2023-06-14 09:37:22.593241
progress:  25.01172424574019 % 2023-06-14 09:37:51.282199
progress:  28.13818977645772 % 2023-06-14 09:38:19.415521
progress:  31.26465530717524 % 2023-06-14 09:38:47.590434
progress:  34.391120837892764 % 2023-06-14 09:39:16.025174
progress:  37.51758636861028 % 2023-06-14 09:39:44.632880
progress:  40.64405189932781 % 2023-06-14 09:40:12.971428
progress:  43.77051743004534 % 2023-06-14 09:40:41.225180
progress:  46.896982960762855 % 2023-06-14 09:41:09.558390
progress:  50.02344849148038 % 2023-06-14 09:41:37.924258
progress:  53.1499140221

In [153]:
with open('output_dialog_processed_tgredial_dict.json', 'w') as outfile:
    json.dump(syn_data, outfile)

In [142]:
with open('output_dialog_processed_tgredial.json', 'w') as outfile:
    json.dump(syn_data, outfile)

In [70]:
len(data)

31985

In [71]:
len(syn_data)

31957

In [6]:
data = json.load(open('output_dialog_processed.json', 'r'))

In [7]:
data.extend(syn_data)

In [9]:
with open('output_dialog_processed_normal.json', 'w') as outfile:
    json.dump(data, outfile)